In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-jan-2021/sample_submission.csv
/kaggle/input/tabular-playground-series-jan-2021/train.csv
/kaggle/input/tabular-playground-series-jan-2021/test.csv


In [2]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error as mse
import xgboost as xg 

In [3]:
df = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2021/train.csv', index_col='id')
df.head()

,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,target
id,,,,,,,,,,,,,,,
1,0.670390,0.811300,0.643968,0.291791,0.284117,0.855953,0.890700,0.285542,0.558245,0.779418,0.921832,0.866772,0.878733,0.305411,7.243043
3,0.388053,0.621104,0.686102,0.501149,0.643790,0.449805,0.510824,0.580748,0.418335,0.432632,0.439872,0.434971,0.369957,0.369484,8.203331
4,0.834950,0.227436,0.301584,0.293408,0.606839,0.829175,0.506143,0.558771,0.587603,0.823312,0.567007,0.677708,0.882938,0.303047,7.776091
5,0.820708,0.160155,0.546887,0.726104,0.282444,0.785108,0.752758,0.823267,0.574466,0.580843,0.769594,0.818143,0.914281,0.279528,6.957716
8,0.935278,0.421235,0.303801,0.880214,0.665610,0.830131,0.487113,0.604157,0.874658,0.863427,0.983575,0.900464,0.935918,0.435772,7.951046


In [4]:
df.shape

(300000, 15)

In [5]:
from tqdm.notebook import tqdm
from sklearn.model_selection import KFold

def estimate_model(pipline, data_df, target):
    k_fold = KFold(n_splits=10, shuffle=False)
    scores = []
    X = data_df.to_numpy()
    y = target.to_numpy()
    for train_index, test_index in tqdm(k_fold.split(X)):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        # Train
        pipline.fit(X_train, y_train)

        # Test
        y_pred = pipline.predict(X_test)
        score = mse(y_test, y_pred, squared=False)
        scores.append(score)
    return scores

In [6]:
X = df.drop('target', axis=1)
y = df['target']

In [7]:
n_test = 10000
x_train = X[:-n_test]
y_train = y[:-n_test]

x_test = X[-n_test:]
y_test = y[-n_test:]

In [8]:
#model = RandomForestRegressor(n_estimators=100, verbose=2)
#model = xg.XGBRegressor(objective ='reg:squarederror', 
#                  n_estimators = 10, seed = 42, n_jobs=-1, verbosity=1)

optimal_params = {'lambda': 0.0030282073258141168, 'alpha': 0.01563845128469084, 'colsample_bytree': 0.5,
             'subsample': 0.7, 'learning_rate': 0.01,'max_depth': 15,
             'random_state': 2020, 'n_estimators': 4000, 'min_child_weight': 257}

#'n_estimators': 4000
model = xg.XGBRegressor(**optimal_params)

In [9]:
model

XGBRegressor(alpha=0.01563845128469084, base_score=None, booster=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.5, gamma=None, gpu_id=None,
             importance_type='gain', interaction_constraints=None,
             lambda=0.0030282073258141168, learning_rate=0.01,
             max_delta_step=None, max_depth=15, min_child_weight=257,
             missing=nan, monotone_constraints=None, n_estimators=4000,
             n_jobs=None, num_parallel_tree=None, random_state=2020,
             reg_alpha=None, reg_lambda=None, scale_pos_weight=None,
             subsample=0.7, tree_method=None, validate_parameters=None,
             verbosity=None)

In [10]:
#scores = estimate_model(model, X, y)

In [11]:
#print("Validation rmse: %0.4f (+/- %0.4f)" % (np.mean(scores), np.std(scores)))

In [12]:
df_test = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2021/test.csv', index_col='id')
df_test.head()

,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14
id,,,,,,,,,,,,,,
0,0.353600,0.738780,0.600939,0.293377,0.285691,0.458006,0.620704,0.422249,0.369203,0.435727,0.550540,0.699134,0.286864,0.364515
2,0.907222,0.189756,0.215531,0.869915,0.301333,0.528958,0.390351,0.521112,0.794779,0.798580,0.446475,0.449037,0.916964,0.513002
6,0.179287,0.355353,0.623972,0.437812,0.282476,0.320826,0.386789,0.776422,0.222268,0.229102,0.211913,0.222651,0.327164,0.827941
7,0.359385,0.181049,0.551368,0.206386,0.280763,0.482076,0.506677,0.362793,0.379737,0.345686,0.445276,0.518485,0.299028,0.598166
10,0.335791,0.682607,0.676481,0.219465,0.282861,0.581721,0.748639,0.350158,0.448915,0.506878,0.817721,0.805895,0.790591,0.249275


In [13]:
model.fit(X, y)

XGBRegressor(alpha=0.01563845128469084, base_score=0.5, booster='gbtree',
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.5,
             gamma=0, gpu_id=-1, importance_type='gain',
             interaction_constraints='', lambda=0.0030282073258141168,
             learning_rate=0.01, max_delta_step=0, max_depth=15,
             min_child_weight=257, missing=nan, monotone_constraints='()',
             n_estimators=4000, n_jobs=0, num_parallel_tree=1,
             random_state=2020, reg_alpha=0.015638452, reg_lambda=0.00302820723,
             scale_pos_weight=1, subsample=0.7, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [14]:
y_pred = model.predict(df_test)

In [15]:
df_test.head()

,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14
id,,,,,,,,,,,,,,
0,0.353600,0.738780,0.600939,0.293377,0.285691,0.458006,0.620704,0.422249,0.369203,0.435727,0.550540,0.699134,0.286864,0.364515
2,0.907222,0.189756,0.215531,0.869915,0.301333,0.528958,0.390351,0.521112,0.794779,0.798580,0.446475,0.449037,0.916964,0.513002
6,0.179287,0.355353,0.623972,0.437812,0.282476,0.320826,0.386789,0.776422,0.222268,0.229102,0.211913,0.222651,0.327164,0.827941
7,0.359385,0.181049,0.551368,0.206386,0.280763,0.482076,0.506677,0.362793,0.379737,0.345686,0.445276,0.518485,0.299028,0.598166
10,0.335791,0.682607,0.676481,0.219465,0.282861,0.581721,0.748639,0.350158,0.448915,0.506878,0.817721,0.805895,0.790591,0.249275


In [16]:
df_sample_submission = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2021/sample_submission.csv')
df_sample_submission.head()

,id,target
0,0,0.5
1,2,0.5
2,6,0.5
3,7,0.5
4,10,0.5


In [17]:
df_sample_submission['target'] = y_pred

In [18]:
df_sample_submission.to_csv('xgboost_optimal_params_4000Nestim.csv', index=False)